In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,260 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main a

In [2]:
from selenium import webdriver # 웹 브라우저 자동화
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time # 시간 지연
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

url = 'https://www.melon.com/chart/week/index.htm?classCd=GN0100'
driver.get(url)
html = driver.page_source
soup = bs(html, 'html.parser')
songs = soup.select('.lst50, .lst100')

In [4]:
data_list = []

In [6]:
for song in songs:
    # 차트 페이지에서의 정보 수집
    title = song.select_one('.ellipsis.rank01 a').text
    rank = song.select_one('.rank').text
    singer = song.select_one('.ellipsis.rank02 a').text
    album = song.select_one('.ellipsis.rank03 a').text
    songid = song.select_one('.like')['data-song-no']

    print(f"Rank: {rank}, Title: {title}, Singer: {singer}, Album: {album}, Song ID: {songid}")

    # 상세 페이지로 이동
    BASE_DETAIL_URL = "https://www.melon.com/song/detail.htm?songId="
    detail_url = BASE_DETAIL_URL + songid
    driver.get(detail_url)
    detail_soup = bs(driver.page_source, 'html.parser')

    # 상세 페이지에서의 정보 수집
    genre = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').find_next_sibling('dd').text.strip()
    release_date = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').text.strip()
    album_image_url = detail_soup.select_one('.thumb .image_typeAll img')['src']
    likes = detail_soup.select_one('#d_like_count').text.strip()
    comments_count = detail_soup.select_one('#revCnt').text.strip()
    lyrics = detail_soup.select_one('.lyric').text.strip()
    composers = []
    lyricists = []
    arrangers = []

    # 각 li 항목을 반복 처리
    for li in detail_soup.select('.list_person.clfix li'):
      artist_name_el = li.select_one('.ellipsis .artist_name')
      artist_type_el = li.select_one('.meta .type')

      if artist_name_el and artist_type_el:  # 두 항목이 모두 존재할 때만 처리
        artist_name = artist_name_el.text.strip()
        artist_type = artist_type_el.text.strip()

        if artist_type == "작곡":
            composers.append(artist_name)
        elif artist_type == "작사":
            lyricists.append(artist_name)
        elif artist_type == "편곡":
            arrangers.append(artist_name)
    print("Composers:", composers)
    print("Lyricists:", lyricists)
    print("Arrangers:", arrangers)

    data_list.append([songid, title, rank, singer, album, likes, release_date, genre, comments_count, lyrics, album_image_url, ", ".join(composers), ", ".join(lyricists), ", ".join(arrangers)])

Rank: 1, Title: 헤어지자 말해요, Singer: 박재정, Album: 1집 Alone, Song ID: 36382580
Composers: ['박재정', '박현중']
Lyricists: ['박재정']
Arrangers: ['박현중']
Rank: 2, Title: 사막에서 꽃을 피우듯, Singer: 우디 (Woody), Album: 사막에서 꽃을 피우듯, Song ID: 36616378
Composers: ['백마리']
Lyricists: ['백마리']
Arrangers: ['전주현']
Rank: 3, Title: 잘 지내자, 우리 (여름날 우리 X 로이킴), Singer: 로이킴, Album: 잘 지내자, 우리 (여름날 우리 X 로이킴), Song ID: 36595401
Composers: ['성용욱']
Lyricists: ['조은영', '성용욱']
Arrangers: ['최상언', '김홍준']
Rank: 4, Title: 나에게 그대만이, Singer: 탑현, Album: 나에게 그대만이, Song ID: 36518341
Composers: ['유해준']
Lyricists: ['경재승']
Arrangers: ['윤일진', '고선생']
Rank: 5, Title: 너의 모든 순간, Singer: 성시경, Album: 별에서 온 그대 OST Part.7, Song ID: 4446485
Composers: ['성시경']
Lyricists: ['심현보']
Arrangers: ['안준영']
Rank: 6, Title: 사랑은 늘 도망가, Singer: 임영웅, Album: 신사와 아가씨 OST Part.2, Song ID: 34061322
Composers: ['홍진영']
Lyricists: ['강태규']
Arrangers: ['한밤 (Midnight)']
Rank: 7, Title: 사랑인가 봐, Singer: 멜로망스, Album: 사랑인가 봐 (사내맞선 OST 스페셜 트랙), Song ID: 34657844
Composers: ['멜로망스']
Ly

In [7]:
# CSV 파일로 저장
import csv
with open('ballad_songs.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["SongID", "Title", "Rank", "Singer", "Album", "Likes", "ReleaseDate", "Genre", "CommentsCount", "Lyrics", "AlbumImageURL", "Composers", "Lyricists", "Arrangers"])
    writer.writerows(data_list)